In [24]:

import plotly.graph_objs as go
import pandas as pd
import numpy as np
import warnings

In [2]:
import plotly

In [3]:
!pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 7.5 MB/s eta 0:00:00


In [4]:
import chart_studio.plotly as py

In [5]:
!pip install plotly_express

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import plotly_express as px

In [15]:
#import the xls file of the tourist dataset
t_inbound = pd.read_excel("/content/travel_data.xls", sheet_name="number_of_arrival", header=3, index_col=0)
t_inbound.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
World,WLD,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.975338e+08,9.563723e+08,9.975553e+08,1.054602e+09,1.106732e+09,1.159450e+09,1.206216e+09,1.250468e+09,1.341457e+09,NaN
High income,HIC,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.483952e+08,5.794189e+08,6.058485e+08,6.320718e+08,6.635699e+08,7.053862e+08,7.386434e+08,7.751152e+08,8.212955e+08,NaN
OECD members,OED,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.016400e+08,5.216800e+08,5.454750e+08,5.637250e+08,5.921553e+08,6.302726e+08,6.641770e+08,6.873700e+08,7.387490e+08,NaN
Post-demographic dividend,PST,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.944450e+08,5.187433e+08,5.428600e+08,5.691869e+08,5.987758e+08,6.194239e+08,6.475276e+08,6.862630e+08,7.311272e+08,NaN
Europe & Central Asia,ECS,"International tourism, number of arrivals",ST.INT.ARVL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.681120e+08,4.805914e+08,5.127784e+08,5.386314e+08,5.677226e+08,5.797844e+08,6.038805e+08,6.159794e+08,6.695441e+08,NaN


In [13]:
#import all the country name list according to the country code
c_name = pd.read_excel("/content/travel_data.xls", sheet_name="country code")
c_name.head()

,Country Name,Country Code
0,Afghanistan,AFG
1,Albania,ALB
2,Algeria,DZA
3,American Samoa,ASM
4,Andorra,AND


In [30]:
#Characterize function to select rows and columns
def data_selection(data):
    #integrate this information framework with the country's list data frame corresponding to the county code so that we can eliminate any redundant rows.
    data = pd.merge(c_name, data, left_on = "Country Code", right_on = "Country Code", how="left")
    data = data.set_index("Country Name")
    #select columns from year 2007 to 2017
    data_new = pd.concat([data["Country Code"],data.loc[:,"2010":"2017"]], axis=1)
    #replace missing value by 0
    data_new = data_new.fillna(0)
    return data_new

In [31]:
#use "data_selection" function to select inbound tourists data
t_inbound_new = data_selection(t_inbound)
t_inbound_new.head()

,Country Code,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,
Afghanistan,AFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,ALB,2191000.0,2469000.0,3156000.0,2857000.0,3341000.0,3784000.0,4070000.0,4643000.0
Algeria,DZA,2070000.0,2395000.0,2634000.0,2733000.0,2301000.0,1710000.0,2039000.0,2451000.0
American Samoa,ASM,23100.0,22600.0,22600.0,20800.0,21600.0,20300.0,20100.0,20000.0
Andorra,AND,1808000.0,2242000.0,2238000.0,2328000.0,2363000.0,2663000.0,2831000.0,0.0


In [64]:
#Characterize function to reshape the data 
def reshape_data(data):
    data_new = data[:] 
    data_new.set_index(["Country Code"], inplace = True, append = True)
    #stack to change the data frame shape
    data_new = data_new.stack()
    data_new = pd.DataFrame(data_new)
    data_new = data_new.reset_index()
    data_new = data_new.rename(columns = {"level_2":"Year",0:"Amount"})
    return data_new

In [65]:
#used the plotly_express package to create the inbound tourists' map.
map = px.choropleth(t_inbound_stack, 
                            locations="Country Code", 
                            color="Amount", 
                            hover_name="Country Name",
                            #set the animation slide bar
                            animation_frame="Year",
                            color_continuous_scale=px.colors.sequential.BuGn,
                            #set the map type
                            projection="natural earth")
map



In [34]:
#use "reshape_data" function to reshape the inbound_touists_clean data frame
t_inbound_stack = reshape_data(t_inbound_new)
t_inbound_stack.head()

,Country Name,Country Code,Year,Amount
0,Afghanistan,AFG,2010,0.0
1,Afghanistan,AFG,2011,0.0
2,Afghanistan,AFG,2012,0.0
3,Afghanistan,AFG,2013,0.0
4,Afghanistan,AFG,2014,0.0


In [37]:
# # Top countries that attract tourists

# In[10]:


#find the top countries that attract tourists
tc_inbound = t_inbound_new.sort_values("2016", ascending=False).head(10)
tc_name_inbound = list(tc_inbound.index)
tc_name_inbound


['France',
 'United States',
 'Spain',
 'China',
 'Italy',
 'United Kingdom',
 'Germany',
 'Mexico',
 'Thailand',
 'Turkey']

In [40]:

#select the top countries' data 
tdata_inbound = t_inbound_stack.loc[t_inbound_stack["Country Name"].isin(tc_name_inbound)]
tdata_inbound.head()


,Country Name,Country Code,Year,Amount
352,China,CHN,2010,55664000.0
353,China,CHN,2011,57581000.0
354,China,CHN,2012,57725000.0
355,China,CHN,2013,55686000.0
356,China,CHN,2014,55622000.0


In [44]:

#line plot for the top countries in the GDP
t_inbound = px.line(tdata_inbound, 
                      x="Year", 
                      y="Amount", 
                      color="Country Name", 
                      line_group="Country Name",  
                      line_shape="linear", 
                      title="Top countries as per tourist reference")
t_inbound


In [46]:
tc_inbound

,Country Code,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,
France,FRA,76647000.0,80499000.0,81980000.0,83634000.0,83701000.0,84452000.0,82682000.0,86861000.0
United States,USA,60010000.0,62821000.0,66657000.0,69995000.0,75022000.0,77774000.0,76407000.0,76941000.0
Spain,ESP,52677000.0,56177000.0,57464000.0,60675000.0,64939000.0,68175000.0,75315000.0,81786000.0
China,CHN,55664000.0,57581000.0,57725000.0,55686000.0,55622000.0,56886000.0,59270000.0,60740000.0
Italy,ITA,43626000.0,46119000.0,46360000.0,47704000.0,48576000.0,50732000.0,52372000.0,58253000.0
United Kingdom,GBR,28295000.0,29306000.0,29282000.0,31063000.0,32613000.0,34436000.0,35814000.0,37651000.0
Germany,DEU,26875000.0,28374000.0,30411000.0,31545000.0,32999000.0,34970000.0,35555000.0,37452000.0
Mexico,MEX,23290000.0,23403000.0,23403000.0,24151000.0,29346000.0,32093000.0,35079000.0,39291000.0
Thailand,THA,15936000.0,19230000.0,22354000.0,26547000.0,24810000.0,29923000.0,32530000.0,35592000.0


In [50]:
# identify the function used to figure out the growth in annual defense expenditures  
def growth_inbound(data):
    for year in range(2010, 2017,1):
        data["growth"+ str(year+1)] = (data[str(year+1)] - data[str(year)])/data[str(year)]*100
        data_new = data.iloc[:, -10:]
    return data_new


In [51]:
growth_data = growth_inbound(tc_inbound)
growth_data = growth_data.stack()
growth_data = growth_data.reset_index()
growth_data = growth_data.rename(columns = {"level_1":"Year",0:"Amount"})
growth_data.head()

,Country Name,Year,Amount
0,France,2015,8.445200e+07
1,France,2016,8.268200e+07
2,France,2017,8.686100e+07
3,France,growth2011,5.025637e+00
4,France,growth2012,1.839774e+00


In [52]:
#construct a bar graph illustrating the rise in visitors from abroad to the whole country. 
bar_growth = px.bar(growth_data, 
                         x="Year", 
                         y="Amount", 
                         color="Country Name", 
                         hover_name="Country Name",                      
                         barmode="group",
                         title="Top tourist-attracting nations' yearly rate of development")
bar_growth


In [53]:
#extract data set for revenue from tourists 
revenue = pd.read_excel("travel_data.xls", sheet_name="receipts for travel items", header=3, index_col=0)
revenue.head()

,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country Name,,,,,,,,,,,,,,,,,,,,,
Aruba,ABW,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.222000e+09,1.251000e+09,1.351000e+09,1.402000e+09,1.495000e+09,1.605000e+09,1.655000e+09,1.625000e+09,1.731000e+09,NaN
Afghanistan,AFG,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.600000e+07,7.500000e+07,8.500000e+07,1.020000e+08,1.380000e+08,1.090000e+08,7.900000e+07,4.900000e+07,2.000000e+06,NaN
Angola,AGO,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.340000e+08,7.190000e+08,6.460000e+08,7.060000e+08,1.234000e+09,1.589000e+09,1.163000e+09,6.230000e+08,8.800000e+08,NaN
Albania,ALB,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.827000e+09,1.611000e+09,1.631000e+09,1.464000e+09,1.473000e+09,1.699000e+09,1.499000e+09,1.692000e+09,1.942000e+09,NaN
Andorra,AND,"International tourism, receipts for travel ite...",ST.INT.TVLR.CD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#select data on visitors who arrive utilizing the "data_selection" method. 
revenue_new = data_selection(revenue)
revenue_new.head()

,Country Code,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,
Afghanistan,AFG,7.500000e+07,8.500000e+07,1.020000e+08,1.380000e+08,1.090000e+08,7.900000e+07,4.900000e+07,2.000000e+06
Albania,ALB,1.611000e+09,1.631000e+09,1.464000e+09,1.473000e+09,1.699000e+09,1.499000e+09,1.692000e+09,1.942000e+09
Algeria,DZA,2.200000e+08,2.090000e+08,2.170000e+08,2.500000e+08,2.580000e+08,3.110000e+08,2.090000e+08,1.410000e+08
American Samoa,ASM,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+07,2.200000e+07
Andorra,AND,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [58]:
#find the top countries with the attract tourists most
tc_revenue = revenue_new.sort_values("2016", ascending=False).head(15)
tc_revenue_name = list(tc_revenue.index)
tc_revenue_name


['United States',
 'Spain',
 'France',
 'Thailand',
 'United Kingdom',
 'China',
 'Italy',
 'Germany',
 'Australia',
 'Hong Kong SAR, China',
 'Japan',
 'Macao SAR, China',
 'India',
 'Mexico',
 'Austria']

In [59]:
#select the top countries' data 
tdata_revenue = revenue_new.loc[revenue_new.index.isin(tc_revenue_name)]
tdata_revenue.head()


,Country Code,2010,2011,2012,2013,2014,2015,2016,2017
Country Name,,,,,,,,,
Australia,AUS,2.847200e+10,3.158200e+10,3.155700e+10,3.061700e+10,3.145200e+10,2.889200e+10,3.474600e+10,4.175400e+10
Austria,AUT,1.875800e+10,1.977500e+10,1.893700e+10,2.022000e+10,2.090700e+10,1.828700e+10,1.923900e+10,2.032300e+10
China,CHN,4.581400e+10,4.846400e+10,5.002800e+10,5.166400e+10,4.404400e+10,4.496900e+10,4.443200e+10,3.261700e+10
France,FRA,4.646600e+10,5.511500e+10,5.334800e+10,5.645700e+10,5.840400e+10,5.830700e+10,5.469000e+10,6.102000e+10
Germany,DEU,3.456400e+10,3.890300e+10,3.806800e+10,4.128500e+10,4.326300e+10,3.688300e+10,3.746900e+10,3.995300e+10


In [61]:
#the revenue_new_stack data frame can be shaped using the "reshape_data" function.
revenue_new_stack = reshape_data(tdata_revenue)
revenue_new_stack.head()


,Country Name,Country Code,Year,Amount
0,Australia,AUS,2010,2.847200e+10
1,Australia,AUS,2011,3.158200e+10
2,Australia,AUS,2012,3.155700e+10
3,Australia,AUS,2013,3.061700e+10
4,Australia,AUS,2014,3.145200e+10


In [62]:
#create a bar graph showing the top tourist-earning nations.
bar_revenue = px.bar(revenue_new_stack, 
                      x="Year", 
                      y="Amount", 
                      color="Country Name", 
                      hover_name="Country Name",
                      title="High revenue Countries")
bar_revenue
